In [1]:
import os
import wfdb
import math
import numpy as np
import pandas as pd
from scipy import interpolate
from scipy import signal
import neurokit2 as nk
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [2]:
# sampling rate: 1000Hz -> resampling rate: 500Hz
# segmentation: 2s -> 1000 timestamps
# the length of the ECG signal recording of each subject can be different
# 15-lead ECGs (12 standard + Frank XYZ leads)
# 290 subjects with 9 diagnostic classes (drop n/a class)

# root path 
root_path = 'ptb-diagnostic-ecg-database-1.0.0'

In [3]:
# select the subject with multiple trials
# less than half of the patients, so we do not drop any patient.
"""
li_sub = []
for sub in os.listdir(root_path):
    num_tri = 0
    sub_path = os.path.join(root_path, sub)
    if os.path.isdir(sub_path):
        for tri in os.listdir(sub_path):
            if '.dat' in tri:
                num_tri += 1
        if num_tri >= 2:
            li_sub.append(sub_path)

print(len(li_sub))

"""

"\nli_sub = []\nfor sub in os.listdir(root_path):\n    num_tri = 0\n    sub_path = os.path.join(root_path, sub)\n    if os.path.isdir(sub_path):\n        for tri in os.listdir(sub_path):\n            if '.dat' in tri:\n                num_tri += 1\n        if num_tri >= 2:\n            li_sub.append(sub_path)\n\nprint(len(li_sub))\n\n"

In [4]:
# only select the patients with myocardial infarction disease label
li_slc_sub = []
for sub in os.listdir(root_path):
    sub_path = os.path.join(root_path, sub)
    if os.path.isdir(sub_path):
        for tri in os.listdir(sub_path):
            if '.dat' in tri:
                tri_path = os.path.join(sub_path, tri)
                label = wfdb.rdsamp(record_name=tri_path[:-4])[1]['comments'][4].split(':')[-1].strip()
                if (label == 'Myocardial infarction')|(label == 'Healthy control'):
                    li_slc_sub.append(sub_path)
                    break

print(len(li_slc_sub))
li_slc_sub

200


['ptb-diagnostic-ecg-database-1.0.0\\patient001',
 'ptb-diagnostic-ecg-database-1.0.0\\patient002',
 'ptb-diagnostic-ecg-database-1.0.0\\patient003',
 'ptb-diagnostic-ecg-database-1.0.0\\patient004',
 'ptb-diagnostic-ecg-database-1.0.0\\patient005',
 'ptb-diagnostic-ecg-database-1.0.0\\patient006',
 'ptb-diagnostic-ecg-database-1.0.0\\patient007',
 'ptb-diagnostic-ecg-database-1.0.0\\patient008',
 'ptb-diagnostic-ecg-database-1.0.0\\patient009',
 'ptb-diagnostic-ecg-database-1.0.0\\patient010',
 'ptb-diagnostic-ecg-database-1.0.0\\patient011',
 'ptb-diagnostic-ecg-database-1.0.0\\patient012',
 'ptb-diagnostic-ecg-database-1.0.0\\patient013',
 'ptb-diagnostic-ecg-database-1.0.0\\patient014',
 'ptb-diagnostic-ecg-database-1.0.0\\patient015',
 'ptb-diagnostic-ecg-database-1.0.0\\patient016',
 'ptb-diagnostic-ecg-database-1.0.0\\patient017',
 'ptb-diagnostic-ecg-database-1.0.0\\patient018',
 'ptb-diagnostic-ecg-database-1.0.0\\patient019',
 'ptb-diagnostic-ecg-database-1.0.0\\patient020',


In [5]:
# resampling to 250Hz
def resampling(array, freq, kind='linear'):
    t = np.linspace(1, len(array), len(array))
    f = interpolate.interp1d(t, array, kind=kind)
    t_new = np.linspace(1, len(array), int(len(array)/freq * 250))
    new_array = f(t_new)
    return new_array

# standard normalization 
def normalize(data):
    scaler = StandardScaler()
    data_norm = scaler.fit_transform(data)
    return data_norm
    
    """
    scaler = MinMaxScaler(feature_range=(0, 1))
    norm = scaler.fit_transform(df.values)
    df_norm = pd.DataFrame(norm)
    return df_norm
    
    """
    

"""
# segmentation with no overlapping (1000 timestamps)
# start from the beginning
def segment(df, window_size=500*10):
    res = []
    index = 0
    while index <= df.shape[0] - window_size:
        res.append(df.iloc[index: index+window_size, :])
        index += window_size
    return res
    
"""


# function of R peaks of a resampled trial
def R_Peaks(ecg_data):
    # get R Peak positions
    pos = []
    # get R Peak intervals
    trial_interval = []
    for ch in range(ecg_data.shape[1]):
        cleaned_ecg = nk.ecg_clean(ecg_data[:, ch], sampling_rate=250, method='neurokit')
        signals, info = nk.ecg_peaks(cleaned_ecg, sampling_rate=250, correct_artifacts=False)
        peaks = signals[signals['ECG_R_Peaks']==1].index.to_list()
        pos.append(peaks)
        channel_interval = []
        for i in range(len(peaks)-1):
            channel_interval.append(peaks[i+1] - peaks[i])
        trial_interval.append(channel_interval)
        
    df_peaks = pd.DataFrame(pos) # [C=15, num of the R-Peaks of a channel]
    df = pd.DataFrame(trial_interval).T
    med = df.median()
    return df, med, df_peaks

In [6]:
# get median R-Peak intervals for all trials
med_intervals = []
li_abnormal_trial = []
for sub_path in li_slc_sub:
    for tri in os.listdir(sub_path):
        if '.dat' in tri:
            tri_path = os.path.join(sub_path, tri)
            ecg_data = wfdb.rdsamp(record_name=tri_path[:-4])[0]
            trial = []
            for ch in range(ecg_data.shape[1]):
                data = resampling(ecg_data[:,ch], freq=1000, kind='linear')
                trial.append(data)
            trial = np.array(trial).T
            trial_norm = normalize(trial)
            try:
                _, med, _ = R_Peaks(trial_norm)
                med_intervals.append(med.to_list())
            except IndexError:
                print('The trial is invalid with trial path {}'.format(tri_path))
                li_abnormal_trial.append(tri_path)
                pass
            
li_abnormal_trial = list(set(li_abnormal_trial))
print(li_abnormal_trial) # no abnormal trial
df_med_intervals = pd.DataFrame(med_intervals).T
df_med_intervals

[]


,0,1,2,3,4,5,6,7,8,9,...,438,439,440,441,442,443,444,445,446,447
0,183.0,174.0,188.0,191.0,208.0,182.0,185.0,157.0,158.0,145.0,...,211.0,160.0,166.0,259.5,179.0,142.0,140.0,203.0,209.0,201.0
1,183.0,177.0,190.0,191.0,208.0,182.0,185.0,157.0,158.0,145.0,...,211.0,160.0,166.0,257.5,179.0,141.0,140.0,202.0,209.0,201.0
2,184.0,174.0,188.0,191.0,207.0,184.0,200.0,157.0,158.0,145.0,...,211.0,160.0,166.0,242.5,179.0,141.0,140.0,202.0,209.0,201.0
3,183.0,174.0,188.0,191.0,208.0,182.0,185.0,157.0,158.0,145.0,...,211.0,160.0,166.0,259.0,179.0,141.0,140.0,202.5,209.0,201.0
4,183.0,174.0,188.0,191.0,208.0,182.0,185.0,158.0,159.0,145.0,...,211.0,160.0,166.0,259.5,179.0,142.0,140.0,203.0,209.0,201.0
5,185.0,174.0,188.0,191.0,208.0,182.0,184.0,157.0,158.0,145.0,...,211.0,159.5,165.0,259.5,179.0,141.0,140.0,202.0,207.0,201.0
6,183.0,174.0,188.0,191.0,208.0,182.0,185.0,157.0,158.5,146.0,...,211.0,160.0,166.0,180.0,178.5,141.0,140.0,203.0,209.0,201.0
7,183.0,174.0,188.0,191.0,207.5,1443.0,1553.0,162.0,163.0,1326.5,...,211.0,160.0,166.0,259.5,179.0,142.0,140.0,197.0,209.0,201.0
8,183.0,174.0,188.0,191.0,208.0,182.0,185.0,519.0,978.0,3074.0,...,211.0,160.0,166.0,259.5,179.0,141.0,140.0,203.0,210.0,201.0
9,183.0,174.0,188.0,191.0,208.0,182.0,185.0,157.0,158.0,17066.0,...,211.0,160.0,166.0,259.5,178.5,142.0,140.0,202.0,209.0,201.0


In [7]:
# set max_duration
all_med = df_med_intervals.median()
print(all_med[all_med<=300].shape)
print(all_med[all_med<=300].max())
max_duration = 300

(438,)
295.0


In [8]:
# update li_abnormal_trial (invalid + outlier)
med_intervals = []
li_abnormal_trial = []
for sub_path in li_slc_sub:
    for tri in os.listdir(sub_path):
        if '.dat' in tri:
            tri_path = os.path.join(sub_path, tri)
            ecg_data = wfdb.rdsamp(record_name=tri_path[:-4])[0]
            trial = []
            for ch in range(ecg_data.shape[1]):
                data = resampling(ecg_data[:,ch], freq=1000, kind='linear')
                trial.append(data)
            trial = np.array(trial).T
            trial_norm = normalize(trial)
            try:
                _, med, _ = R_Peaks(trial_norm)
                if med.median() <= max_duration: 
                    med_intervals.append(med.to_list())
                else:
                    print('The trial is an outlier with trial path {}'.format(tri_path))
                    li_abnormal_trial.append(tri_path)
            except IndexError:
                print('The trial is invalid with trial path {}'.format(tri_path))
                li_abnormal_trial.append(tri_path)
                pass
            
li_abnormal_trial = list(set(li_abnormal_trial))
print(li_abnormal_trial) # no abnormal trial
df_med_intervals = pd.DataFrame(med_intervals).T
df_med_intervals

The trial is an outlier with trial path ptb-diagnostic-ecg-database-1.0.0\patient023\s0085lre.dat
The trial is an outlier with trial path ptb-diagnostic-ecg-database-1.0.0\patient023\s0103lre.dat
The trial is an outlier with trial path ptb-diagnostic-ecg-database-1.0.0\patient028\s0108lre.dat
The trial is an outlier with trial path ptb-diagnostic-ecg-database-1.0.0\patient085\s0297lre.dat
The trial is an outlier with trial path ptb-diagnostic-ecg-database-1.0.0\patient085\s0298lre.dat
The trial is an outlier with trial path ptb-diagnostic-ecg-database-1.0.0\patient273\s0511_re.dat
The trial is an outlier with trial path ptb-diagnostic-ecg-database-1.0.0\patient277\s0527_re.dat
The trial is an outlier with trial path ptb-diagnostic-ecg-database-1.0.0\patient279\s0532_re.dat
The trial is an outlier with trial path ptb-diagnostic-ecg-database-1.0.0\patient279\s0533_re.dat
The trial is an outlier with trial path ptb-diagnostic-ecg-database-1.0.0\patient279\s0534_re.dat
['ptb-diagnostic-ecg

,0,1,2,3,4,5,6,7,8,9,...,428,429,430,431,432,433,434,435,436,437
0,183.0,174.0,188.0,191.0,208.0,182.0,185.0,157.0,158.0,145.0,...,211.0,160.0,166.0,259.5,179.0,142.0,140.0,203.0,209.0,201.0
1,183.0,177.0,190.0,191.0,208.0,182.0,185.0,157.0,158.0,145.0,...,211.0,160.0,166.0,257.5,179.0,141.0,140.0,202.0,209.0,201.0
2,184.0,174.0,188.0,191.0,207.0,184.0,200.0,157.0,158.0,145.0,...,211.0,160.0,166.0,242.5,179.0,141.0,140.0,202.0,209.0,201.0
3,183.0,174.0,188.0,191.0,208.0,182.0,185.0,157.0,158.0,145.0,...,211.0,160.0,166.0,259.0,179.0,141.0,140.0,202.5,209.0,201.0
4,183.0,174.0,188.0,191.0,208.0,182.0,185.0,158.0,159.0,145.0,...,211.0,160.0,166.0,259.5,179.0,142.0,140.0,203.0,209.0,201.0
5,185.0,174.0,188.0,191.0,208.0,182.0,184.0,157.0,158.0,145.0,...,211.0,159.5,165.0,259.5,179.0,141.0,140.0,202.0,207.0,201.0
6,183.0,174.0,188.0,191.0,208.0,182.0,185.0,157.0,158.5,146.0,...,211.0,160.0,166.0,180.0,178.5,141.0,140.0,203.0,209.0,201.0
7,183.0,174.0,188.0,191.0,207.5,1443.0,1553.0,162.0,163.0,1326.5,...,211.0,160.0,166.0,259.5,179.0,142.0,140.0,197.0,209.0,201.0
8,183.0,174.0,188.0,191.0,208.0,182.0,185.0,519.0,978.0,3074.0,...,211.0,160.0,166.0,259.5,179.0,141.0,140.0,203.0,210.0,201.0
9,183.0,174.0,188.0,191.0,208.0,182.0,185.0,157.0,158.0,17066.0,...,211.0,160.0,166.0,259.5,178.5,142.0,140.0,202.0,209.0,201.0


In [9]:
# split resampled trial to sample level(single heartbeat)
def sample(ecg_data, max_duration=300):
    samples = []
    _, med, df_peaks = R_Peaks(ecg_data)
    trial_med = med.median()
    for i in range(df_peaks.shape[1]):
        RP_pos = df_peaks.iloc[:, i].median()
        ini_beat = ecg_data[max(0,int(RP_pos)-int(trial_med/2)):min(int(RP_pos)+int(trial_med/2),ecg_data.shape[0]), :]
        left_zero_num = int((int(max_duration)-ini_beat.shape[0])/2)
        padding_left = np.zeros([left_zero_num, ecg_data.shape[1]])
        padding_right = np.zeros([int(max_duration)-left_zero_num-ini_beat.shape[0], ecg_data.shape[1]])
        beat = np.concatenate([padding_left, ini_beat, padding_right], axis=0)
        samples.append(beat)
    return samples 


# concat samples to segmentations
def sample2seg(samples, seg_size=10):
    segmentations = []
    index = 0
    while index <= len(samples)-seg_size:
        beat = samples[index]
        for i in range(index+1, index+seg_size):
            beat = np.vstack((beat, samples[i]))
        segmentations.append(beat)
        index += seg_size
    return segmentations

In [10]:
# main
feature_path = './Feature'
if not os.path.exists(feature_path):
    os.mkdir(feature_path)

dict_label = {}
sub_id = 1
for sub_path in li_slc_sub:
    li_sub_segs = []
    for tri in os.listdir(sub_path):
        if 'dat' in tri:
            tri_path = os.path.join(sub_path, tri)
            if tri_path not in li_abnormal_trial:
                label = wfdb.rdsamp(record_name=tri_path[:-4])[1]['comments'][4].split(':')[-1].strip() # label
                if label == 'Myocardial infarction':
                    dict_label['{}'.format(sub_id)] = 1
                if label == 'Healthy control':
                    dict_label['{}'.format(sub_id)] = 0
                ecg_data = wfdb.rdsamp(record_name=tri_path[:-4])[0] # data
                trial = []
                for ch in range(ecg_data.shape[1]):
                    data = resampling(ecg_data[:,ch], freq=1000, kind='linear')
                    trial.append(data)
                trial = np.array(trial).T
                trial_norm = normalize(trial)
                samples = sample(trial_norm, max_duration=300)
                segmentations = sample2seg(samples, seg_size=10) # all segmentations of a trial
                for seg in segmentations:
                    li_sub_segs.append(seg)
                    print(seg.shape)
                    
    if li_sub_segs != list(): # Not None list
        array_sub = np.array(li_sub_segs)
        print(array_sub.shape)
        print('\n')
        np.save(feature_path + '/feature_{:03d}.npy'.format(sub_id), array_sub)
        sub_id += 1
    else:
        print('The subject is None after preprocessing with the path {}'.format(tri_path))              

(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(36, 3000, 15)


(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(15, 3000, 15)


(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(13, 3000, 15)


(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(3000, 15)
(30

In [11]:
# test feature_X_Y.npy
np.load('./Feature/feature_183.npy').shape

(21, 3000, 15)

In [12]:
# label.npy
label_path = './Label'
if not os.path.exists(label_path):
    os.mkdir(label_path)

df_label = pd.DataFrame([dict_label]).T
df_label = df_label.reset_index().astype('int64')
labels = df_label[[0, 'index']].values
np.save(label_path + '/label.npy', labels)

In [13]:
# test label.npy
np.load('./Label/label.npy')

array([[  1,   1],
       [  1,   2],
       [  1,   3],
       [  1,   4],
       [  1,   5],
       [  1,   6],
       [  1,   7],
       [  1,   8],
       [  1,   9],
       [  1,  10],
       [  1,  11],
       [  1,  12],
       [  1,  13],
       [  1,  14],
       [  1,  15],
       [  1,  16],
       [  1,  17],
       [  1,  18],
       [  1,  19],
       [  1,  20],
       [  1,  21],
       [  1,  22],
       [  1,  23],
       [  1,  24],
       [  1,  25],
       [  1,  26],
       [  1,  27],
       [  1,  28],
       [  1,  29],
       [  1,  30],
       [  1,  31],
       [  1,  32],
       [  1,  33],
       [  1,  34],
       [  1,  35],
       [  1,  36],
       [  1,  37],
       [  1,  38],
       [  1,  39],
       [  1,  40],
       [  1,  41],
       [  1,  42],
       [  1,  43],
       [  1,  44],
       [  1,  45],
       [  1,  46],
       [  1,  47],
       [  1,  48],
       [  1,  49],
       [  1,  50],
       [  1,  51],
       [  1,  52],
       [  1,